In [2]:
import numpy as np
import random
import itertools 
import os
from collections import defaultdict
import json

In [3]:
import random
import numpy as np
from time import time
from tqdm import tqdm

In [15]:
import numpy as np
from tqdm import tqdm

red = ['0'] * 26
black = ['1'] * 26
deck = black + red

np.random.seed(123)

results = []
for i in tqdm(range(10000)):
    np.random.shuffle(deck)
    results += [int(''.join(deck),2)]

np.save('results_10000', results)

100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 73275.62it/s]


## Reading in Npy 

In [6]:
def load_process_simulations(file_path): 
    simulations = np.load(file_path)
    
    def int_to_bit(n):
        binary = bin(n)[2:] #what? 
        return binary.zfill(52)
    
    binary_simulations = [int_to_bit(sim) for sim in simulations]             
    
    return binary_simulations             

In [4]:
processed_sims = load_process_simulations('./penney-game/simulation/results.npy')


In [16]:
processed_sims = load_process_simulations('results_10000.npy')


In [31]:
simulations = np.load('sim1.npy')

In [32]:
simulations #why is the sim1 npy file different? 

array([4354504303719388, 1441862137223036, 3132395670338690,
       3857300131579263,  831690148884508,  187288135370219,
       1931508377813441, 3900754009753688, 3784952613550390,
        762196189488984])

In [17]:
def variation1(deck, player1_sequence, player2_sequence):
    player1_cards = 0
    player2_cards = 0
    pile = 0
    
    for i in range(len(deck) - 2):
        current_sequence = deck[i:i+3]
        pile += 1
        if current_sequence == player1_sequence:
            player1_cards += pile
            pile = 0
        elif current_sequence == player2_sequence:
            player2_cards += pile
            pile = 0
    #what happens to cards in the pile at end of the game? 
    
    return player1_cards, player2_cards

In [21]:
def pattern_game_52bits(n, player1_pattern, player2_pattern):
    binary_str = bin(n)[2:].zfill(52)
    
    player1_points = 0
    player2_points = 0
    
    bits_traversed = 0
    last_pattern_position = 0
    
    i = 0
    while i <= len(binary_str) - 3: 
        current_bits = binary_str[i:i+3]
        bits_traversed += 1
        
        if current_bits == player1_pattern:
            # add 2 because we are looking at 3 bits at a time
            player1_points += bits_traversed - last_pattern_position + 2
            last_pattern_position = bits_traversed
            i += 2 #overlapping patterns ????????
        elif current_bits == player2_pattern:
            player2_points += bits_traversed - last_pattern_position + 2
            last_pattern_position = bits_traversed
            i += 2
        i += 1
    
    return player1_points, player2_points

In [18]:
def analyze_all_combinations(processed_sims):
    all_sequences = [''.join(seq) for seq in itertools.product('01', repeat=3)]
    results = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
               for p1 in all_sequences for p2 in all_sequences if p1 != p2}
    
    for deck in processed_sims:
        for (p1_seq, p2_seq), result in results.items():
            p1_cards, p2_cards = variation1(deck, p1_seq, p2_seq)
            if p1_cards > p2_cards:
                result['player1_wins'] += 1
            elif p2_cards > p1_cards:
                result['player2_wins'] += 1
    
    return results

In [19]:
%%time
results = analyze_all_combinations(processed_sims)
results

CPU times: user 7.13 s, sys: 12.9 ms, total: 7.14 s
Wall time: 7.26 s


{('000', '001'): {'player1_wins': 4789, 'player2_wins': 4987},
 ('000', '010'): {'player1_wins': 2889, 'player2_wins': 6907},
 ('000', '011'): {'player1_wins': 1891, 'player2_wins': 7842},
 ('000', '100'): {'player1_wins': 3, 'player2_wins': 9997},
 ('000', '101'): {'player1_wins': 2687, 'player2_wins': 7075},
 ('000', '110'): {'player1_wins': 201, 'player2_wins': 9741},
 ('000', '111'): {'player1_wins': 4875, 'player2_wins': 4777},
 ('001', '000'): {'player1_wins': 4987, 'player2_wins': 4789},
 ('001', '010'): {'player1_wins': 8380, 'player2_wins': 1422},
 ('001', '011'): {'player1_wins': 9312, 'player2_wins': 556},
 ('001', '100'): {'player1_wins': 6, 'player2_wins': 9992},
 ('001', '101'): {'player1_wins': 8319, 'player2_wins': 1433},
 ('001', '110'): {'player1_wins': 4693, 'player2_wins': 4708},
 ('001', '111'): {'player1_wins': 9720, 'player2_wins': 220},
 ('010', '000'): {'player1_wins': 6907, 'player2_wins': 2889},
 ('010', '001'): {'player1_wins': 1422, 'player2_wins': 8380},
 

In [20]:
string_keyed_results = {str(key): value for key, value in results.items()}


In [21]:
with open('10000_sims_results', 'w') as outfile:
    json.dump(string_keyed_results, outfile)